In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf

In [2]:
df=pd.read_excel('PraeterBV_Case.xlsx',sheet_name='Opdracht_2_input_output')
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Toepassing,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,Algemeen,NaN,NaN,NaN
3,NaN,Inflatie,0.02,NaN,inputs
4,NaN,Afschrijving,Lineair,NaN,output
5,NaN,Eigen Vermogen,0.2,NaN,Stamgegevens
6,NaN,Rente VV,0.04,NaN,NaN
7,NaN,Belasting,0.165,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [3]:
def cashflowsRekening(data, output_path="cashflow_output2.xlsx"):
    df = pd.read_excel(data,sheet_name='Opdracht_2_input_output')

    investering = df.iloc[14, 2]
    subsidie_eenmalig = df.iloc[15, 2]
    restwaarde = df.iloc[16, 2]

    print(f"Investering: {investering}")
    print(f"Subsidie(eenmalig): {subsidie_eenmalig}")
    print(f"Restwaarde: {restwaarde}")

    besparing = df.iloc[19, 2]
    subsidie_jaarlijks = df.iloc[20, 2]
    belasting = df.iloc[7, 2]

    print(f"Besparing: {besparing}")
    print(f"Subsidie(jaarlijks): {subsidie_jaarlijks}")
    print(f"Belasting: {belasting}")

    kosten_eenmalig = df.iloc[23, 2]
    kosten_exploitatie = df.iloc[24, 2]
    herinvestering = df.iloc[25, 2]

    print(f"Eenmalige kosten: {kosten_eenmalig}")
    print(f"Vaste exploitatiekosten: {kosten_exploitatie}")
    print(f"Herinvestering: {herinvestering}")

    termijn = int(df.iloc[11, 2])
    inflatie = df.iloc[3, 2]
    jaar_herinvesteer = int(df.iloc[12, 2])
    Totale_investering = df.iloc[28, 2]
    afschrijving = df.iloc[29, 2]
    Aflossing = df.iloc[30, 2]

    print(f"Termijn: {termijn}")
    print(f"Inflatie: {inflatie}")
    print(f"Jaar herinvestering: {jaar_herinvesteer}")
    print(f"Aflossing: {Aflossing}")
    print(f"Afschrijving: {afschrijving}")
    print(f"Totale investering: {Totale_investering}")

    eigen_vermogen = Totale_investering * 0.2 + kosten_eenmalig
    print(f"Eigen vermogen: {eigen_vermogen}")

    rente_VV = [0] + [int((120000 - 10000 * (i - 1)) * 0.04) for i in range(1, termijn + 1)]

    belasting_perc = 0.1650
    cashflows_rev = [-eigen_vermogen]
    winst_na_belasting = [kosten_eenmalig]
    start_investering = -(investering - subsidie_eenmalig + kosten_eenmalig)
    cashflows = [start_investering]

    for jaar in range(1, termijn + 1):
        inkomsten = (besparing + subsidie_jaarlijks) * ((1 + inflatie) ** (jaar - 1))
        kosten = kosten_exploitatie * ((1 + inflatie) ** (jaar - 1))

        if jaar == jaar_herinvesteer:
            kosten += herinvestering

        ebitda = inkomsten - kosten
        ebit = ebitda - afschrijving
        rente = rente_VV[jaar]
        ebt = ebit - rente
        belasting = ebt * belasting_perc if ebt > 0 else 0
        pat = ebt - belasting

        cashflow = pat + afschrijving + rente
        cashflows.append(round(cashflow, 0))

        cashflow_rev = (pat + afschrijving) - Aflossing
        cashflows_rev.append(round(cashflow_rev, 0))

        winst_na_belasting.append(round(pat, 0))

    print("\n Jaarlijkse cashflow IRR:")
    for jaar, cf in enumerate(cashflows):
        print(f"jaar {jaar}: {cf:,.0f} ")

    irr = npf.irr(cashflows) * 100
    print(f"(IRR): {irr:.2f}%")

    print("\n Jaarlijkse cashflow REV:")
    for jaar, rev in enumerate(cashflows_rev):
        print(f"REV jaar {jaar}: {rev:,.0f}")

    rev = npf.irr(cashflows_rev) * 100
    print(f"(REV): {rev:.2f}%")

    print("\n Winst na belasting:")
    for jaar, winst in enumerate(winst_na_belasting):
        print(f"jaar {jaar}: {winst:,.0f}")

    totaal_winst = sum(winst_na_belasting)
    winst_netto = totaal_winst - eigen_vermogen
    print(f"Winst na belasting: {winst_netto:,.0f}")

    cumulative = 0
    tvt = 0
    for i in range(1, len(cashflows_rev)):
        cumulative += cashflows_rev[i]
        if cumulative >= eigen_vermogen:
            overschrijden = cumulative - eigen_vermogen
            overshoot_fractional = overschrijden / cashflows_rev[i]
            tvt = i - overshoot_fractional
            break

    print(f"\n TVT = {tvt:.2f} jaar")

    jaren = list(range(0, termijn + 1))
    df_output = pd.DataFrame({
        "Jaar": jaren,
        "Cashflow IRR (€)": cashflows,
        "Cashflow REV (€)": cashflows_rev,
        "Winst na belasting (€)": winst_na_belasting
    })

    extra_info = pd.DataFrame({
        "Jaar": ["", "IRR (%)", "REV (%)", "Winst Netto (€)", "TVT (jaren)"],
        "Cashflow IRR (€)": ["", round(irr, 2), "", round(winst_netto, 0), round(tvt, 2)],
        "Cashflow REV (€)": ["", "", round(rev, 2), "", ""],
        "Winst na belasting (€)": ["", "", "", "", ""]
    })

    df_final = pd.concat([df_output, extra_info], ignore_index=True)
    df_final.to_excel(output_path, index=False)

    print(f"\n Bestand succesvol opgeslagen: {output_path}")


cashflowsRekening('PraeterBV_Case.xlsx', output_path="output_cashflows.xlsx")


Investering: 160000
Subsidie(eenmalig): 10000
Restwaarde: 0
Besparing: 20000
Subsidie(jaarlijks): 800
Belasting: 0.165
Eenmalige kosten: 30000
Vaste exploitatiekosten: 2000
Herinvestering: 4000
Termijn: 12
Inflatie: 0.02
Jaar herinvestering: 6
Aflossing: 10000
Afschrijving: 12500
Totale investering: 150000
Eigen vermogen: 60000.0

 Jaarlijkse cashflow IRR:
jaar 0: -180,000 
jaar 1: 18,552 
jaar 2: 18,800 
jaar 3: 19,055 
jaar 4: 19,315 
jaar 5: 19,583 
jaar 6: 16,516 
jaar 7: 20,137 
jaar 8: 20,425 
jaar 9: 20,719 
jaar 10: 21,021 
jaar 11: 21,330 
jaar 12: 21,647 
(IRR): 4.39%

 Jaarlijkse cashflow REV:
REV jaar 0: -60,000
REV jaar 1: 3,752
REV jaar 2: 4,400
REV jaar 3: 5,055
REV jaar 4: 5,715
REV jaar 5: 6,383
REV jaar 6: 3,716
REV jaar 7: 7,737
REV jaar 8: 8,425
REV jaar 9: 9,119
REV jaar 10: 9,821
REV jaar 11: 10,530
REV jaar 12: 11,247
(REV): 4.98%

 Winst na belasting:
jaar 0: 30,000
jaar 1: 1,252
jaar 2: 1,900
jaar 3: 2,555
jaar 4: 3,215
jaar 5: 3,883
jaar 6: 1,216
jaar 7: 5,237